# ABS Monthly Consumer Price Index

## Python set up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import matplotlib.patheffects as pe
import pandas as pd
import readabs as ra
from readabs import metacol as mc

# local imports
from abs_helper import (
    collate_summary_data,
    get_abs_data,
    ANNUAL_CPI_TARGET_RANGE,
    MONTHLY_CPI_TARGET,
    QUARTERLY_CPI_TARGET,
    QUARTERLY_CPI_RANGE,
)
from mgplot import (
    finalise_plot,
    line_plot_finalise,
    series_growth_plot_finalise,
    summary_plot_finalise,
    calc_growth
)

from henderson import hma

In [2]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

YEARLY_RANGE = {"axhspan": ANNUAL_CPI_TARGET_RANGE}
MONTHLY_TARGET = {"axhline": MONTHLY_CPI_TARGET}

# display charts in this notebook
SHOW = False
FILE_TYPE = "png"

## Data capture

In [3]:
abs_dict, meta, source, RECENT = get_abs_data("6401.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

In [4]:
meta[mc.table].unique()

array(['640101', '640102', '640103', '640104', '640105', '640106',
       '640107', '640108', '640109', '6401010', '6401011', '6401012',
       '6401013', '6401014', '6401015', '6401016'], dtype=object)

## Plotting

### Headline growth annual/monthly

In [5]:
def recent_headnline(recent:int=19):
    """Plot the main monthly CPI series.
    
    Args:
        recent (int): Number of recent months to plot. Defaults to 18.
    """

    wanted = [
        {"Table": '640106', "DID": "Index Numbers ;  All groups CPI ;  Australia ;"},
        {"Table": '640106', "DID": "Index Numbers ;  All groups CPI, seasonally adjusted ;  Australia ;"},
        {"Table": '640106', "DID": "Index Numbers ;  Trimmed Mean ;  Australia ;"},
        {"Table": '640106', "DID": "Index Numbers ;  Weighted Median ;  Australia ;"},
    ]

    for w in wanted:
        selector = {
            w["Table"]: mc.table,
            w["DID"]: mc.did,
        }
        table, series_id, units = ra.find_abs_id(meta, selector)
        series = abs_dict[table][series_id]
        left = meta[(meta[mc.id] == series_id)&(meta[mc.table] == table)][mc.stype].values[0]
        label = w["DID"].replace("Index Numbers ;", "").replace(" ;", "").replace("  ", " ").strip()
        label: str = label if "CPI" in label else f"CPI - {label}"
        series_growth_plot_finalise(
            series.iloc[-recent:],
            title=label,
            lfooter = f"Australia. Monthly data. Calculated from the raw index numbers. {left} series. ",
            rfooter = source,
            bar_rounding=2,
            line_rounding=1,
            **YEARLY_RANGE,
            **MONTHLY_TARGET,
            show=SHOW,
            file_type=FILE_TYPE,
        )


recent_headnline()

In [6]:
def recent_core_annualised(n=6) -> None:
    """Plot recent annualised monthly core CPI growth.
    
    Args:
        n (int): Number of recent months to plot. Defaults to 6.
    """

    wanted = [
        {"Table": '640106', "DID": "Index Numbers ;  Trimmed Mean ;  Australia ;"},
        {"Table": '640106', "DID": "Index Numbers ;  Weighted Median ;  Australia ;"},
    ]

    box = pd.DataFrame()
    for w in wanted:
        selector = {
            w["Table"]: mc.table,
            w["DID"]: mc.did,
        }
        table, series_id, units = ra.find_abs_id(meta, selector)
        series = abs_dict[table][series_id]
        growth = series.pct_change(n) + 1
        annualised = (growth ** (12 / n)) - 1
        name = "Trimmed Mean" if "Trimmed" in w["DID"] else "Weighted Median"
        box[name] = annualised * 100
    
    line_plot_finalise(
        box.iloc[-19:],
        title=f"Recent {n}-Month Annualised Core CPI Growth",
        lfooter = f"Australia. Monthly data. Calculated from the raw index numbers. ",
        rfooter = source,
        annotate=True,
        **YEARLY_RANGE,
        show=SHOW,
        file_type=FILE_TYPE,
    )


recent_core_annualised(n=6)
recent_core_annualised(n=4)
recent_core_annualised(n=3)

## Finished

In [7]:
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-11-26 13:47:35

Python implementation: CPython
Python version       : 3.14.0
IPython version      : 9.7.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 25.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

matplotlib: 3.10.7
readabs   : 0.1.5
pandas    : 2.3.3
mgplot    : 0.2.12

Watermark: 2.5.0



In [8]:
print("End of Notebook")

End of Notebook
